# Convert RA and DEC to a Unit Direction in Heliocentric Frame

### Known Observation of Ceres According to JPL

In [1]:
# library imports
import numpy as np
import astropy
from astropy.coordinates import SkyCoord, GCRS, ICRS, HCRS, HeliocentricMeanEcliptic, HeliocentricTrueEcliptic, EarthLocation
from astropy.units import deg, au, meter

#  MSE imports
import astro_utils
import asteroid_integrate
import asteroid_data
from asteroid_data import make_data_one_file, get_earth_pos

In [2]:
# MJD of observation time
obstime_mjd = 58600.0

# Convert this to a JD
obstime_jd = astro_utils.mjd_to_jd(obstime_mjd)
print(f'Observation Time as JD: {obstime_jd}')

Observation Time as JD: 2458600.5


In [3]:
# RA and DEC of Ceres at 2458600.5 according to JPL
ra_deg = 252.250075738
dec_deg = -17.009538673
delta_au = 1.86800464519883

# Heliocentric coordinates of Ceres
ast_hel_lon_deg = 240.239114
ast_hel_lat_deg = 3.671885
ast_r_au = 2.738704926

# Heliocentric coordinates of Earth
earth_hel_lon_deg = 216.1950
earth_hel_lat_deg = 0.0017  
earth_r_au = 1.006347536397

In [4]:
# Reference frames
# https://docs.astropy.org/en/stable/api/astropy.coordinates.GCRS.html
# https://docs.astropy.org/en/stable/api/astropy.coordinates.ICRS.html
frame_earth = GCRS
frame_solar = HeliocentricMeanEcliptic

In [5]:
# Earth location of the Palomar observatory
geoloc_palomar_auto = EarthLocation.of_site('Palomar')
print(f'geolocation of Palomar observatory:')
print(f'geoloc_palomar = {geoloc_palomar_auto}')
print(f'geoloc_palomar geodetic = {geoloc_palomar_auto.geodetic}')

geolocation of Palomar observatory:
geoloc_palomar = (-2410346.78217658, -4758666.82504051, 3487942.97502457) m
geoloc_palomar geodetic = GeodeticLocation(lon=<Longitude -116.863 deg>, lat=<Latitude 33.356 deg>, height=<Quantity 1706. m>)


In [6]:
# For some reason, the 'nice' way to do this below leads to a bug in constructor to GCRS
# geoloc_palomar = EarthLocation(-2410346.8, -4758666.8, 3487943, unit='m')

# Workaround: extract the geolocation of the Palomar observatory using EarthLocation.of_site
geoloc_palomar = [-2410346.78217658, -4758666.82504051, 3487942.97502457] * meter

### Coordinates of Ceres at MJD 58600.5 as SkyCoord instance

In [7]:
# c = SkyCoord(ra=ra_deg, dec=dec_deg, unit='deg', frame=frame_earth)

In [8]:
# c = SkyCoord(ra=ra_deg*astropy.units.deg, dec=dec_deg*astropy.units.deg, distance=delta_au*astropy.units.au, frame=frame_earth)

In [9]:
# c = SkyCoord(ra=ra_deg, dec=dec_deg, distance=delta_au, unit='deg', frame=frame_earth)

In [10]:
# Convert from numbers to class instances

# RA and DEC of asteroid in earth frame
ra = ra_deg * deg
dec = dec_deg * deg
delta = delta_au * au

# Asteroid in solar frame
ast_hel_lon = ast_hel_lon_deg * deg
ast_hel_lat = ast_hel_lat_deg * deg
ast_r = ast_r_au * au

# Earth in solar frame
earth_hel_lon = earth_hel_lon_deg * deg
earth_hel_lat = earth_hel_lat_deg * deg
earth_r = earth_r_au * au

# Distances of zero and one AU
zero_au = 0.0 * au 
one_au = 1.0 * au

# Angle of zero degrees
zero_deg = 0.0 * deg

# Observation time
obstime = astropy.time.Time(val=obstime_mjd, format='mjd')

# Display class outputs
print(f'Observation of Ceres according to JPL:')
print(f'obstime  = {repr(obstime)}')
print(f'ra       = {ra}')
print(f'dec      = {dec}')
print(f'delta    = {delta}')
print(f'hEcl lon = {ast_hel_lon}')
print(f'hEcl lat = {ast_hel_lat}')
print(f'ast r    = {ast_r}')

Observation of Ceres according to JPL:
obstime  = <Time object: scale='utc' format='mjd' value=58600.0>
ra       = 252.250075738 deg
dec      = -17.009538673 deg
delta    = 1.86800464519883 AU
hEcl lon = 240.239114 deg
hEcl lat = 3.671885 deg
ast r    = 2.738704926 AU


In [11]:
# Coordinates of Ceres in geocentric frame; include observation time & location.  
# Use correct distance delta_au from JPL
# ast_geo = GCRS(ra=ra, dec=dec, distance=distance, obstime=obstime, obsgeoloc=geoloc_palomar)
ast_geo = SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=frame_earth)
ast_geo

<SkyCoord (GCRS: obstime=58600.0, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (252.25007574, -17.00953867, 1.86800465)>

In [12]:
# Coordinates of Earth in geocentric frame; this is easy, distance is zero!
# earth_geo = GCRS(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, obsgeoloc = geoloc_palomar)
earth_geo = SkyCoord(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, frame=frame_earth)
earth_geo

<SkyCoord (GCRS: obstime=58600.0, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (0., 0., 0.)>

### Unit direction of Ceres in Heliocentric (J2000) Frame from RA and Dec

In [13]:
# Relative displacement from Earth to Ceres in the Heliocentric frame
# This is critical, need to transform from ICRS to Heliocentric for axes to be oriented correctly
ast_hel = ast_geo.transform_to(frame_solar)
earth_hel = earth_geo.transform_to(frame_solar)
rel_hel = (ast_hel.cartesian - earth_hel.cartesian)
print(f'Heliocentric Cartesian coordinates from JPL RA / DEC Data:')
print('Ceres:   ', ast_hel.cartesian)
print('Earth:   ', earth_hel.cartesian)
print('Relative:', rel_hel)

Heliocentric Cartesian coordinates from JPL RA / DEC Data:
Ceres:    (-1.35679056, -2.37255796, 0.17540414) AU
Earth:    (-0.81207743, -0.59436424, 3.01115264e-05) AU
Relative: (-0.54471313, -1.77819372, 0.17537403) AU


In [14]:
# Extract arrays from heliocentric objects
ast_jpl = ast_hel.cartesian.xyz.value
earth_jpl = earth_hel.cartesian.xyz.value
rel_jpl = rel_hel.xyz.value

In [15]:
# Direction from earth to asteroid
u_jpl = rel_jpl / np.linalg.norm(rel_jpl)
print('Direction from Earth to Ceres:')
np.round(u_jpl, 6)

Direction from Earth to Ceres:


array([-0.291602, -0.951921,  0.093883])

In [16]:
# Calculation in a single step using coordinate transformations.
# Don't need the distance, just the RA and the Dec!
ast_geo2 = SkyCoord(ra=ra, dec=dec, distance=one_au, obstime=obstime, frame=HCRS)
u_jpl2 = ast_geo2.transform_to(frame_solar).cartesian.xyz.value
u_jpl2 = u_jpl2 / np.linalg.norm(u_jpl2)
norm_diff = np.linalg.norm(u_jpl2 - u_jpl)
print(f'Difference between u_jpl and u_jpl2 method with HCRS frame:')
print(f'Norm of difference =  {norm_diff:8.6f} = {np.rad2deg(norm_diff):5.3f} degrees')

Difference between u_jpl and u_jpl2 method with HCRS frame:
Norm of difference =  0.000078 = 0.004 degrees


### Conclusion: Conversion Procedure
We can convert from an RA and DEC to a unit direction in two lines of code by:<br>
1) Create the initial observation as a SkyCoord instance in the **HCRS frame** (Heloiocentric, axis aligned to GCRS)<br>
2) Convert this observation to the **HeliocentricMeanEcliptic frame**<br>
The two coordinate systems share the same origin (the sun, NOT the solar system barycenter).<br>
The only difference is the axis alignment; the axes in the HCRS are aligned with earth's equator / north pole, while the axes in HeliocentricMeanEcliptic are have the z-axis orthogonal to the ecliptic, with the x-axis pointing to the equinox.<br>
Experiments show that the same procedure using either ICRS or GCRS are not consistent with the explicit two step procedure.

### Unit Direction of Ceres Using Heliocentric Latitude & Longitude

In [17]:
# Heliocentric coordinates of Ceres and Earth
ast_hel2 = SkyCoord(lon=ast_hel_lon, lat=ast_hel_lat, distance=ast_r, obstime=obstime, frame=frame_solar)
earth_hel2 = SkyCoord(lon=earth_hel_lon, lat=earth_hel_lat, distance=earth_r, obstime=obstime, frame=frame_solar)

# Cartesian vectors
ast_jpl2 = ast_hel2.cartesian.xyz.value
earth_jpl2 = earth_hel2.cartesian.xyz.value
rel_jpl2 = ast_jpl2 - earth_jpl2
u_jpl2 = rel_jpl2 / np.linalg.norm(rel_jpl2)

In [18]:
print(f'Heliocentric Cartesian coordinates from JPL Heliocentric Coordinate Data:')
print('Ceres:    ', ast_hel2.cartesian)
print('Earth:    ', earth_hel2.cartesian)
print('Relative: ', rel_jpl2)
print('Direction:', u_jpl2)

Heliocentric Cartesian coordinates from JPL Heliocentric Coordinate Data:
Ceres:     (-1.35665161, -2.37260154, 0.17539385) AU
Earth:     (-0.81213439, -0.59428369, 2.98589325e-05) AU
Relative:  [-0.54451722 -1.77831785  0.17536399]
Direction: [-0.29148734 -0.95195729  0.09387469]


### Compare two methods of Using JPL Data to Generate Directions

In [19]:
# Compare two methods 
diff_ast = ast_jpl2 - ast_jpl
diff_earth = earth_jpl2 - earth_jpl
diff_rel = rel_jpl2 - rel_jpl
diff_u = u_jpl2 - u_jpl
diff_u_norm = np.linalg.norm(diff_u)

print(f'Difference in position (heliocentric long/lat - RA/DEC)')
print(f'Asteroid  : norm={np.linalg.norm(diff_ast):5.2e} AU')
print(f'Earth     : norm={np.linalg.norm(diff_earth):5.2e} AU')
print(f'Rel       : norm={np.linalg.norm(diff_rel):5.2e} AU')
print(f'Direction : norm={diff_u_norm:5.2e} / {np.rad2deg(diff_u_norm):6.4f} degrees')
print(f'Asteroid: ', diff_ast)
print('Earth    :', diff_earth)
print('Rel      :', diff_rel)

Difference in position (heliocentric long/lat - RA/DEC)
Asteroid  : norm=1.46e-04 AU
Earth     : norm=9.87e-05 AU
Rel       : norm=2.32e-04 AU
Direction : norm=1.20e-04 / 0.0069 degrees
Asteroid:  [ 1.38948737e-04 -4.35846551e-05 -1.02891127e-05]
Earth    : [-5.69603076e-05  8.05489055e-05 -2.52593919e-07]
Rel      : [ 1.95909044e-04 -1.24133561e-04 -1.00365188e-05]


**Conclusion**<br>
These results are quite close, but not identical.<br>
Positions are different on the order of $10^{-4}$ AU.<br>
The direction on the unit sphere is different on the order of 0.0069 degrees (28 seconds)

### MSE Integrated Coordinates of Ceres at MJD  58600

In [20]:
ast_elt = asteroid_integrate.load_data()

In [21]:
inputs, outputs = make_data_one_file(0, 1000)

In [22]:
inputs.keys()

dict_keys(['a', 'e', 'inc', 'Omega', 'omega', 'f', 'epoch', 'ts'])

In [23]:
outputs.keys()

dict_keys(['q', 'v', 'u'])

In [24]:
astro_utils.mjd_to_date(int(obstime_mjd))

datetime.date(2019, 4, 27)

In [25]:
astro_utils.mjd_to_datetime(obstime_mjd)

datetime.datetime(2019, 4, 27, 0, 0)

In [26]:
ast_elt

,Num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541124,541124,2018 RP23,58600.0,2.586399,0.289358,0.088749,2.000720,3.913328,1.075531,17.30,0.15,JPL 7,1.654537,1519.293350,0.004136,0.706394,1.285400,-0.369137,-260.066715
541125,541125,2018 RV23,58600.0,3.113036,0.213678,0.203046,0.544794,0.242079,0.130760,16.10,0.15,JPL 8,0.206083,2006.201725,0.003132,0.917633,0.992956,0.786873,-41.751113
541126,541126,2018 RP24,58600.0,2.453880,0.176693,0.194504,2.649626,3.695880,0.937231,17.30,0.15,JPL 6,1.258854,1404.036362,0.004475,0.999551,1.321174,0.062320,-209.433076


In [27]:
# Index for Ceres
idx_ast = np.where(ast_elt.Name=='Ceres')

# Get the index for observation time
ts = inputs['ts'][idx_ast].flatten()

In [28]:
ts

array([51544., 51545., 51546., ..., 66517., 66518., 66519.], dtype=float32)

In [29]:
# Index of observation time <= obs_time
idx_t = np.searchsorted(ts, obstime_mjd-0.5)

In [30]:
t0 = ts[idx_t].item()
t1 = ts[idx_t+1].item()
w0 = (t1 - obstime_mjd) / (t1 - t0 )
w1 = (obstime_mjd - t0) / (t1 - t0)
print(f't0:      ', t0)
print(f't1:      ', t1)
print(f'obstime, ', obstime_mjd)
print(f'w0: ', w0)
print(f'w1: ', w1)

t0:       58600.0
t1:       58601.0
obstime,  58600.0
w0:  1.0
w1:  0.0


In [31]:
# Interpolated asteroid positions as observation time
q0 = outputs['q'][:, idx_t]
q1 = outputs['q'][:, idx_t+1]
q = w0 * q0 + w1 * q1

In [32]:
# Predicted position and direction of Ceres
ast_mse = q[idx_ast].flatten()
np.round(ast_mse,6)

array([-1.356567, -2.37266 ,  0.175377], dtype=float32)

In [33]:
# Get position of Earth using utility function
earth_mse = get_earth_pos(obstime_mjd)

In [34]:
np.round(earth_mse, 6)

array([-8.12085e-01, -5.94353e-01,  3.00000e-05])

In [35]:
# Manually compute displacement from Earth to Ceres
rel_mse = ast_mse - earth_mse
np.round(rel_mse, 6)

array([-0.544482, -1.778307,  0.175346])

In [36]:
# Manually compute the unit direction
u_mse2 = rel_mse / np.linalg.norm(rel_mse)
np.round(u_mse2, 6)

array([-0.291472, -0.951963,  0.093866])

In [37]:
# MSE direction via lookup of output field u
u0 = outputs['u'][:, idx_t]
u1 = outputs['u'][:, idx_t+1]
u = w0 * u0 + w1 * u1
u_mse = u[idx_ast].flatten()
np.round(u_mse, 6)

array([-0.291472, -0.951963,  0.093866], dtype=float32)

In [38]:
# Demonstrate that two methods of getting direction vector from MSE essentially identical
np.linalg.norm(u_mse - u_mse2)

2.9161358092572046e-08

In [39]:
print(f'Heliocentric Cartesian coordinates from MSE Integration:')
print('Ceres:   ', ast_mse)
print('Earth:   ', earth_mse)
print('Relative:', rel_mse)
print('Direction:', u_mse)

Heliocentric Cartesian coordinates from MSE Integration:
Ceres:    [-1.3565673  -2.3726602   0.17537652]
Earth:    [-8.12085390e-01 -5.94353020e-01  3.01103355e-05]
Relative: [-0.54448187 -1.77830714  0.17534641]
Direction: [-0.2914719  -0.9519628   0.09386639]


**Conclusion**<br>
We can look up the calculated direction from Earth to an asteroid in one line by:<br>
1) Load the asteroid direction data by calling <br>
```inputs, outputs = make_data_one_file(0, 1000)```<br>
2) Extract the times as ```inputs['ts'][idx_ast]``` where ```idx_ast``` is the index of the desired asteroid<br>
3) Extract the computed directions as ```outputs['u'][idx_ast]```<br>
4) Compute predicted direction by interpolating the desired time (as an MJD)<br>
These results are consistent with manually subtracting the integrated asteroid position minus the earth's position in the heliocentric frame.

### Compare MSE Calculations to JPL Using RA / DEC

In [40]:
ast_err = ast_mse - ast_jpl
ast_err_norm = np.linalg.norm(ast_err)
print('ast_err: ', ast_err)
print(f'norm ast_err = {ast_err_norm:5.2e}')

ast_err:  [ 2.23294013e-04 -1.02201840e-04 -2.76193442e-05]
norm ast_err = 2.47e-04


In [41]:
earth_err = earth_mse - earth_jpl
earth_err_norm = np.linalg.norm(earth_err)
print('earth_err: ', earth_err)
print(f'norm earth_err = {earth_err_norm:5.2e}')

earth_err:  [-7.96453230e-06  1.12172120e-05 -1.19089308e-09]
norm earth_err = 1.38e-05


In [42]:
rel_err = rel_mse - rel_jpl
rel_err_norm = np.linalg.norm(rel_err)
print('rel_err: ', rel_err)
print(f'norm rel_err = {rel_err_norm:5.2e}')

rel_err:  [ 2.31258545e-04 -1.13419052e-04 -2.76181533e-05]
norm rel_err = 2.59e-04


In [43]:
u_err = u_mse - u_jpl
u_err_norm = np.linalg.norm(u_err)
print('direction_err: ', u_err)
print(f'norm direction_err = {u_err_norm:5.2e}')
print(f'in degrees = {np.rad2deg(u_err_norm):6.4f} = ({np.rad2deg(u_err_norm)*3600:4.1f} seconds)')

direction_err:  [ 1.29695400e-04 -4.13623850e-05 -1.66933205e-05]
norm direction_err = 1.37e-04
in degrees = 0.0079 = (28.3 seconds)


**Conclusion**<br>
Directions predicted by MSE integration are consistent with results from JPL to within 0.0079 degrees / 28 seconds.<br>

**Compare MSE Calculations to JPL Using Heliocentric Long/Lat**

In [44]:
ast_err2 = ast_mse - ast_jpl2
ast_err_norm2 = np.linalg.norm(ast_err2)
print('ast_err2: ', ast_err2)
print(f'norm ast_err2 = {ast_err_norm2:5.2e}')

ast_err2:  [ 8.43452762e-05 -5.86171847e-05 -1.73302315e-05]
norm ast_err2 = 1.04e-04


In [45]:
earth_err2 = earth_mse - earth_jpl2
earth_err_norm2 = np.linalg.norm(earth_err2)
print('earth_err2: ', earth_err2)
print(f'norm earth_err2 = {earth_err_norm2:5.2e}')

earth_err2:  [ 4.89957753e-05 -6.93316934e-05  2.51403026e-07]
norm earth_err2 = 8.49e-05


In [46]:
rel_err2 = rel_mse - rel_jpl2
rel_err2_norm = np.linalg.norm(rel_err2)
print('rel_err2: ', rel_err2)
print(f'norm rel_err2 = {rel_err2_norm:5.2e}')

rel_err2:  [ 3.53495010e-05  1.07145088e-05 -1.75816345e-05]
norm rel_err2 = 4.09e-05


In [47]:
u_err2 = u_mse - u_jpl2
u_err2_norm = np.linalg.norm(u_err2)
print('direction_err: ', u_err2)
print(f'norm direction_err2 = {u_err2_norm:5.2e}')
print(f'in degrees = {np.rad2deg(u_err2_norm):6.4f} = ({np.rad2deg(u_err2_norm)*3600:4.1f} seconds)')

direction_err:  [ 1.54397328e-05 -5.54225966e-06 -8.29973751e-06]
norm direction_err2 = 1.84e-05
in degrees = 0.0011 = ( 3.8 seconds)


**Conclusion**<br>
Errors using heliocentric long / lat are significantly smaller than using direct RA / DEC transformation.<br>
MSE calculation is consistent to the JPL unit direction derived from heliocentric coordinates to within 3.8 arc seconds!